This script is used to fun DNN and CNN model in google colab and do 5-fold cross validation.
Since typically a deep learning model would take huge time to train,we utilize colan free GPU to speed up. Meanwhile, a trained keras model is easy to move to R environment.

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/ESE529-project") 
os.listdir()

['final_train_data.RData',
 'train_price.RData',
 'true_price.RData',
 '__pycache__',
 'utils.py',
 'DNN (1).py',
 'dnn_model1.h5',
 'run_keras_model.ipynb']

In [0]:
#import data from drive
import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [0]:
robjects.r.load('final_train_data.RData')
final_train_data = robjects.r['final_train_data']
final_train_data=np.array(final_train_data)

In [0]:
final_train_data.shape

(1481661, 188)

In [0]:
robjects.r.load('true_price.RData')
true_price = robjects.r['true_price']
true_price=np.array(true_price)

In [0]:
train_price=np.log(true_price+1)

In [0]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 123

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

Using TensorFlow backend.


In [0]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input,Dense,BatchNormalization,Activation,Conv1D,\
    Flatten,Reshape,Dropout,concatenate,MaxPooling1D,add
from tensorflow.keras import Model
from tensorflow.keras import regularizers

#dnn model function
def DNN(input_dim,layer_list,activation="relu",dropout=0.2):
    input=Input((input_dim,))
    for i in range(len(layer_list)):
        if i==0:
            dense=Dense(layer_list[i],activation=activation,
                        kernel_regularizer=regularizers.l2(0.001))(input)
            dense=BatchNormalization()(dense)
            dense=Activation(activation)(dense)
            if dropout>0:
                dense=Dropout(dropout)(dense)
        else:
            dense=Dense(layer_list[i],activation=activation,
                        kernel_regularizer=regularizers.l2(0.001))(dense)
            dense=BatchNormalization()(dense)
            dense=Activation(activation)(dense)
            if dropout>0:
                dense=Dropout(dropout)(dense)
    output=Dense(1)(dense)
    DNN_model=Model(input,output)
    return DNN_model


In [0]:
#dnn modle with residual 
def DNN_residual(input_dim,dropout=0.2):
     input=Input((input_dim,))
     #feature 1
     dense1_input=Dense(256,activation="relu",kernel_regularizer=regularizers.l2(0.001))(input)
     dense1=Dense(512,activation="relu",kernel_regularizer=regularizers.l2(0.001))(dense1_input)
     dense1=Dense(1024,activation="relu",kernel_regularizer=regularizers.l2(0.001))(dense1)
     dense1=Dense(256,activation="relu",kernel_regularizer=regularizers.l2(0.001))(dense1)
     dense1=add([dense1_input,dense1])
     dense1=BatchNormalization()(dense1)
     dense1=Activation("relu")(dense1)
     dense1=Dropout(dropout)(dense1)

    #feature 2
     dense2_input=Dense(512,activation="relu",kernel_regularizer=regularizers.l2(0.001))(input)
     dense2=Dense(1024,activation="relu",kernel_regularizer=regularizers.l2(0.001))(dense2_input)
     dense2=Dense(2048,activation="relu",kernel_regularizer=regularizers.l2(0.001))(dense2)
     dense2=Dense(512,activation="relu",kernel_regularizer=regularizers.l2(0.001))(dense2)
     dense2=add([dense2_input,dense2])
     dense2=BatchNormalization()(dense2)
     dense2=Activation("relu")(dense2)
     dense2=Dropout(dropout)(dense2)

     final_dense=concatenate([dense1,dense2])
     final_dense=Dense(256,activation="relu",kernel_regularizer=regularizers.l2(0.001))(final_dense)
     final_dense=Dropout(dropout)(final_dense)
     final_dense=Dense(32)(final_dense)

     dnn_residual=Model(input,final_dense)
     return dnn_residual

In [0]:
#inception module
def inception(filter_size):

    def f(input_layer):
        inception1=Conv1D(filter_size,1,padding="same",activation="relu")(input_layer)

        inception2=Conv1D(filter_size,1,padding="same",activation="relu")(input_layer)
        inception2=Conv1D(filter_size,3,padding="same",activation="relu")(inception2)

        inception3=Conv1D(filter_size,1,padding="same",activation="relu")(input_layer)
        inception3=Conv1D(filter_size,3,padding="same",activation="relu")(inception3)
        inception3=Conv1D(filter_size,3,padding="same",activation="relu")(inception3)

        inception4=MaxPooling1D(padding="same",strides=1)(input_layer)
        inception4=Conv1D(filter_size,3,padding="same",activation="relu")(inception4)
        concat=concatenate([inception1,inception2,inception3,inception4],axis=-1)
        concat=Conv1D(filter_size,1,padding="same",activation="relu")(concat)

        output_layer=add([input_layer,concat])
        return output_layer
    return f

#cnn model function
def CNN(input_dim1,input_dim2,filter_list1,dropout=0.2,reg=0.0001):
    catgorical_input=Input((input_dim1,))
    catgorical_dense=Dense(256,activation="relu")(catgorical_input)
    catgorical_dense=Dense(512,activation="relu")(catgorical_dense)

    text_input=Input((input_dim2,))
    text_reshape=Reshape((input_dim2,1))(text_input)
    for i in range(len(filter_list1)):
        if i == 0:

            text_conv = Conv1D(filter_list1[i], 5, padding="same", activation="relu",kernel_regularizer=regularizers.l2(reg))(text_reshape)
        else:
            text_conv = Conv1D(filter_list1[i], 5, padding="same", activation="relu",kernel_regularizer=regularizers.l2(reg))(text_conv)
        text_conv = inception(filter_list1[i])(text_conv)
        text_conv = BatchNormalization()(text_conv)
        text_conv = Activation("relu")(text_conv)
        text_conv = Dropout(dropout)(text_conv)

    text_flatten=Flatten()(text_conv)
    print(K.int_shape(text_flatten))
    concat=concatenate([catgorical_dense,text_flatten])
    
    new_shape=K.int_shape(concat)[1]
    concat=Reshape((new_shape,1))(concat)
    for j in range(len(filter_list2)):
        if j == 0:
            conv = Conv1D(filter_list2[j], 5, padding="same", activation="relu",kernel_regularizer=regularizers.l2(reg))(concat)
        else:
            conv = Conv1D(filter_list2[j], 5, padding="same", activation="relu",kernel_regularizer=regularizers.l2(reg))(conv)
        conv = inception(filter_list2[j])(conv)
        conv = BatchNormalization()(conv)
        conv = Activation("relu")(conv)
        conv = Dropout(dropout)(conv)
    final_dense=Flatten()(conv)
    
    final_dense=Dense(2048,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    final_dense=Dense(1024,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    final_dense=Dense(256,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    final_dense=Dense(64,activation="relu",kernel_regularizer=regularizers.l2(reg))(final_dense)
    output=Dense(1)(final_dense)

    CNN_model=Model([catgorical_input,text_input],output)
    return CNN_model



In [0]:
def mse(y_pre,y_tr):
    original_y_pre=np.exp(y_pre)-1
    num_sample=y_pre.shape[0]
    mse_result=np.sum(np.square(original_y_pre-y_tr))/num_sample
    return mse_result

def rmsle(y_pre,y_tr):
    nat_y_tr=np.log(y_tr+1)
    num_sample=y_pre.shape[0]
    rmsle_result=np.sqrt(np.sum(np.square(y_pre-nat_y_tr))/num_sample)
    return rmsle_result

In [0]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [0]:
#split train and test
train_x,test_x,train_y,test_y,train_true_y,test_true_y=train_test_split(final_train_data,train_price,true_price,test_size=0.3)

In [0]:
#clear memory 
del final_train_data
import gc
gc.collect()

22

In [0]:
#since colab only have limited memory, we manually do cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=seed_value)
train_mse_list=[]
val_mse_list=[]
train_rmsle_list=[]
val_rmsle_list=[]
index_list=list(kfold.split(train_x,train_y))

In [0]:
#here we only retain one fold for demonstrating.
train,val=index_list[0]
dnn_model=DNN(188,[512,1024,2048,512,256,64],dropout=0.2)
dnn_model.compile(loss="mse", 
                optimizer='adam',
                metrics=['mse'])
dnn_model.fit(train_x[train,:],train_y[train],batch_size=2048,epochs=25,validation_data=(train_x[val,:],train_y[val]))

Epoch 1/200
406/406 [==============================] - 8s 19ms/step - loss: 5.0957 - mse: 1.8070 - val_loss: 3.6408 - val_mse: 0.9367
Epoch 2/200
406/406 [==============================] - 7s 18ms/step - loss: 2.7987 - mse: 0.7119 - val_loss: 2.0206 - val_mse: 0.4935
Epoch 3/200
406/406 [==============================] - 7s 18ms/step - loss: 1.6818 - mse: 0.5613 - val_loss: 1.2288 - val_mse: 0.4434
Epoch 4/200
406/406 [==============================] - 7s 18ms/step - loss: 1.0645 - mse: 0.4942 - val_loss: 0.8383 - val_mse: 0.4400
Epoch 5/200
406/406 [==============================] - 7s 18ms/step - loss: 0.7467 - mse: 0.4538 - val_loss: 0.6028 - val_mse: 0.3930
Epoch 6/200
406/406 [==============================] - 7s 18ms/step - loss: 0.5883 - mse: 0.4284 - val_loss: 0.5038 - val_mse: 0.3839
Epoch 7/200
406/406 [==============================] - 7s 18ms/step - loss: 0.5085 - mse: 0.4121 - val_loss: 0.4504 - val_mse: 0.3729
Epoch 8/200
406/406 [==============================] - 7s 18ms

KeyboardInterrupt: ignored

In [0]:
#since colab only have limited memory, we manually do cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=seed_value)
train_mse_list=[]
val_mse_list=[]
train_rmsle_list=[]
val_rmsle_list=[]
index_list=list(kfold.split(train_x,train_y))

In [0]:

train,val=index_list[0]
cnn_model=CNN(132,56,[8,32,64],[4,8,16])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([train_x[train,:132],train_x[train,132:]],train_y[train],batch_size=2048,epochs=200,validation_data=([train_x[val,:132],train_x[val,132:]],train_y[val]))


(None, 3584)
Epoch 1/200
406/406 [==============================] - 34s 84ms/step - loss: 2.0223 - mse: 1.5653 - val_loss: 0.8279 - val_mse: 0.4060
Epoch 2/200
406/406 [==============================] - 33s 82ms/step - loss: 0.7826 - mse: 0.3939 - val_loss: 0.7308 - val_mse: 0.3729
Epoch 3/200
406/406 [==============================] - 33s 81ms/step - loss: 0.7111 - mse: 0.3794 - val_loss: 0.6952 - val_mse: 0.3876
Epoch 4/200
406/406 [==============================] - 33s 81ms/step - loss: 0.6557 - mse: 0.3694 - val_loss: 0.6379 - val_mse: 0.3713
Epoch 5/200
406/406 [==============================] - 33s 81ms/step - loss: 0.6097 - mse: 0.3608 - val_loss: 0.5989 - val_mse: 0.3663
Epoch 6/200
406/406 [==============================] - 33s 81ms/step - loss: 0.5712 - mse: 0.3535 - val_loss: 0.5786 - val_mse: 0.3744
Epoch 7/200
406/406 [==============================] - 33s 81ms/step - loss: 0.5401 - mse: 0.3484 - val_loss: 0.5249 - val_mse: 0.3451
Epoch 8/200
406/406 [=====================

In [0]:
train_result=cnn_model.predict([train_x[train,:132],train_x[train,132:]])
val_result=cnn_model.predict([train_x[val,:132],train_x[val,132:]])

In [0]:
train_mse=mse(train_result,train_true_y[train])
train_rmsle=rmsle(train_result,train_true_y[train])
val_mse=mse(val_result,train_true_y[val])
val_rmsle=rmsle(val_result,train_true_y[val])
train_mse_list.append(train_mse)
val_mse_list.append(val_mse)
train_rmsle_list.append(train_rmsle)
val_rmsle_list.append(val_rmsle)

In [0]:
val_rmsle

0.5463470840583688

In [0]:
train,val=index_list[1]
cnn_model=CNN(132,56,[8,32,64],[4,8,16])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([train_x[train,:132],train_x[train,132:]],train_y[train],batch_size=2048,epochs=100,validation_data=([train_x[val,:132],train_x[val,132:]],train_y[val]))

(None, 3584)
Epoch 1/100
406/406 [==============================] - 33s 82ms/step - loss: 1.8705 - mse: 1.4196 - val_loss: 0.8365 - val_mse: 0.4286
Epoch 2/100
406/406 [==============================] - 33s 81ms/step - loss: 0.7620 - mse: 0.3914 - val_loss: 0.7204 - val_mse: 0.3832
Epoch 3/100
406/406 [==============================] - 33s 81ms/step - loss: 0.6861 - mse: 0.3765 - val_loss: 0.6511 - val_mse: 0.3663
Epoch 4/100
406/406 [==============================] - 33s 81ms/step - loss: 0.6289 - mse: 0.3652 - val_loss: 0.5952 - val_mse: 0.3509
Epoch 5/100
406/406 [==============================] - 33s 81ms/step - loss: 0.5857 - mse: 0.3583 - val_loss: 0.5656 - val_mse: 0.3537
Epoch 6/100
406/406 [==============================] - 33s 81ms/step - loss: 0.5498 - mse: 0.3517 - val_loss: 0.5366 - val_mse: 0.3514
Epoch 7/100
406/406 [==============================] - 33s 81ms/step - loss: 0.5211 - mse: 0.3475 - val_loss: 0.5251 - val_mse: 0.3624
Epoch 8/100
406/406 [=====================

In [0]:
train_result=cnn_model.predict([train_x[train,:132],train_x[train,132:]])
val_result=cnn_model.predict([train_x[val,:132],train_x[val,132:]])

In [0]:
train_mse=mse(train_result,train_true_y[train])
train_rmsle=rmsle(train_result,train_true_y[train])
val_mse=mse(val_result,train_true_y[val])
val_rmsle=rmsle(val_result,train_true_y[val])
train_mse_list.append(train_mse)
val_mse_list.append(val_mse)
train_rmsle_list.append(train_rmsle)
val_rmsle_list.append(val_rmsle)

In [0]:
val_rmsle

[962.1196778333715, 1023.1848829278421]

In [0]:
train,val=index_list[2]
cnn_model=CNN(132,56,[8,32,64],[4,8,16])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([train_x[train,:132],train_x[train,132:]],train_y[train],batch_size=2048,epochs=100,validation_data=([train_x[val,:132],train_x[val,132:]],train_y[val]))

(None, 3584)
Epoch 1/100
406/406 [==============================] - 34s 83ms/step - loss: 1.3334 - mse: 0.8857 - val_loss: 0.8383 - val_mse: 0.4377
Epoch 2/100
406/406 [==============================] - 33s 80ms/step - loss: 0.7451 - mse: 0.3855 - val_loss: 0.7163 - val_mse: 0.3933
Epoch 3/100
406/406 [==============================] - 33s 80ms/step - loss: 0.6620 - mse: 0.3699 - val_loss: 0.6214 - val_mse: 0.3572
Epoch 4/100
406/406 [==============================] - 33s 80ms/step - loss: 0.6033 - mse: 0.3627 - val_loss: 0.5637 - val_mse: 0.3446
Epoch 5/100
406/406 [==============================] - 33s 80ms/step - loss: 0.5558 - mse: 0.3553 - val_loss: 0.5251 - val_mse: 0.3414
Epoch 6/100
406/406 [==============================] - 32s 80ms/step - loss: 0.5183 - mse: 0.3495 - val_loss: 0.4913 - val_mse: 0.3360
Epoch 7/100
406/406 [==============================] - 33s 80ms/step - loss: 0.4879 - mse: 0.3448 - val_loss: 0.4722 - val_mse: 0.3402
Epoch 8/100
406/406 [=====================

In [0]:
train_result=cnn_model.predict([train_x[train,:132],train_x[train,132:]])
val_result=cnn_model.predict([train_x[val,:132],train_x[val,132:]])

In [0]:
train_mse=mse(train_result,train_true_y[train])
train_rmsle=rmsle(train_result,train_true_y[train])
val_mse=mse(val_result,train_true_y[val])
val_rmsle=rmsle(val_result,train_true_y[val])
train_mse_list.append(train_mse)
val_mse_list.append(val_mse)
train_rmsle_list.append(train_rmsle)
val_rmsle_list.append(val_rmsle)

In [0]:
val_rmsle

0.548929705520856

In [0]:
print(train_rmsle_list)
print(val_rmsle_list)
print(train_mse_list)
print(val_mse_list)

[0.5208154900823853, 0.5256938382109262, 0.525401342055649]
[0.5463470840583688, 0.5478289952287667, 0.548929705520856]
[962.1196778333715, 1023.1848829278421, 999.4979940776974]
[1051.1171796139038, 1129.3330861106715, 1069.9572244904255]


In [0]:
train,val=index_list[3]
cnn_model=CNN(132,56,[8,32,64],[4,8,16])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([train_x[train,:132],train_x[train,132:]],train_y[train],batch_size=2048,epochs=100,validation_data=([train_x[val,:132],train_x[val,132:]],train_y[val]))

(None, 3584)
Epoch 1/100
406/406 [==============================] - 34s 84ms/step - loss: 1.7427 - mse: 1.2823 - val_loss: 0.8418 - val_mse: 0.4167
Epoch 2/100
406/406 [==============================] - 33s 81ms/step - loss: 0.7812 - mse: 0.3893 - val_loss: 0.7268 - val_mse: 0.3662
Epoch 3/100
406/406 [==============================] - 33s 81ms/step - loss: 0.7071 - mse: 0.3733 - val_loss: 0.6600 - val_mse: 0.3519
Epoch 4/100
406/406 [==============================] - 33s 81ms/step - loss: 0.6528 - mse: 0.3665 - val_loss: 0.6100 - val_mse: 0.3451
Epoch 5/100
406/406 [==============================] - 33s 81ms/step - loss: 0.6042 - mse: 0.3574 - val_loss: 0.5698 - val_mse: 0.3407
Epoch 6/100
406/406 [==============================] - 33s 81ms/step - loss: 0.5649 - mse: 0.3510 - val_loss: 0.5350 - val_mse: 0.3361
Epoch 7/100
406/406 [==============================] - 33s 81ms/step - loss: 0.5324 - mse: 0.3463 - val_loss: 0.5092 - val_mse: 0.3361
Epoch 8/100
406/406 [=====================

In [0]:
train_result=cnn_model.predict([train_x[train,:132],train_x[train,132:]])
val_result=cnn_model.predict([train_x[val,:132],train_x[val,132:]])

In [0]:
train_mse=mse(train_result,train_true_y[train])
train_rmsle=rmsle(train_result,train_true_y[train])
val_mse=mse(val_result,train_true_y[val])
val_rmsle=rmsle(val_result,train_true_y[val])
train_mse_list.append(train_mse)
val_mse_list.append(val_mse)
train_rmsle_list.append(train_rmsle)
val_rmsle_list.append(val_rmsle)

In [0]:
val_rmsle

0.5462582733200518

In [0]:
train,val=index_list[4]
cnn_model=CNN(132,56,[8,32,64],[4,8,16])
cnn_model.compile(loss='mse', 
                optimizer='adam',
                metrics=['mse'])
cnn_model.fit([train_x[train,:132],train_x[train,132:]],train_y[train],batch_size=2048,epochs=100,validation_data=([train_x[val,:132],train_x[val,132:]],train_y[val]))

(None, 3584)
Epoch 1/100
406/406 [==============================] - 33s 81ms/step - loss: 1.8317 - mse: 1.3804 - val_loss: 0.8209 - val_mse: 0.4127
Epoch 2/100
406/406 [==============================] - 32s 80ms/step - loss: 0.7635 - mse: 0.3924 - val_loss: 0.7064 - val_mse: 0.3693
Epoch 3/100
406/406 [==============================] - 32s 80ms/step - loss: 0.6873 - mse: 0.3771 - val_loss: 0.6476 - val_mse: 0.3627
Epoch 4/100
406/406 [==============================] - 33s 80ms/step - loss: 0.6313 - mse: 0.3670 - val_loss: 0.5991 - val_mse: 0.3547
Epoch 5/100
406/406 [==============================] - 33s 80ms/step - loss: 0.5882 - mse: 0.3605 - val_loss: 0.5639 - val_mse: 0.3522
Epoch 6/100
406/406 [==============================] - 33s 80ms/step - loss: 0.5526 - mse: 0.3544 - val_loss: 0.5289 - val_mse: 0.3438
Epoch 7/100
406/406 [==============================] - 32s 80ms/step - loss: 0.5209 - mse: 0.3473 - val_loss: 0.5017 - val_mse: 0.3394
Epoch 8/100
406/406 [=====================

In [0]:
train_result=cnn_model.predict([train_x[train,:132],train_x[train,132:]])
val_result=cnn_model.predict([train_x[val,:132],train_x[val,132:]])

In [0]:
train_mse=mse(train_result,train_true_y[train])
train_rmsle=rmsle(train_result,train_true_y[train])
val_mse=mse(val_result,train_true_y[val])
val_rmsle=rmsle(val_result,train_true_y[val])
train_mse_list.append(train_mse)
val_mse_list.append(val_mse)
train_rmsle_list.append(train_rmsle)
val_rmsle_list.append(val_rmsle)

In [0]:
val_rmsle

0.5456448186580519

In [0]:
np.mean(train_mse_list)

1041.6227243039853

In [0]:
np.mean(train_rmsle_list)

0.5778004352760864

In [0]:
np.mean(val_rmsle_list)

0.581350693692059

In [0]:
np.mean(val_mse_list)

1163.0134331843622